In [24]:
# tests to find solution for https://github.com/c-scale-community/use-case-hisea/issues/28

from datetime import timedelta, datetime
from pathlib import Path
import subprocess
import xarray as xr # note dependencies: dask, netCDF4

vars = ('thetao', 'bottomT', 'so', 'zos', 'uo', 'vo')
date_min = '2022-04-01'
date_max = '2022-04-05'

delta = datetime.strptime(date_max, '%Y-%m-%d') - datetime.strptime(date_min, '%Y-%m-%d')

var = vars[0]
i = 0
day = datetime.strptime(date_min, '%Y-%m-%d').date() + timedelta(days=i)
check_file = Path('/Users/backeb/Documents/data/tmp/cmems_'+str(var)+'_'+str(day)+'.nc')

j=0
while not check_file.is_file():
    print('j '+str(i))
    print(check_file.is_file())
    open(check_file, 'a')
    j+=1

print(check_file.is_file())
print(j)
    

j 0
False
True
1
